### Part 0: Import and Download NCBI Taxa Database

reference:http://etetoolkit.org/docs/latest/tutorial/tutorial_ncbitaxonomy.html

In [47]:
import numpy as np
import pandas as pd
from ete3 import NCBITaxa

In [48]:
# Download NCBI Taxa Database
ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()

### Part 1: Read Nanopore Data

In [49]:
raw = pd.read_csv('../nanopore_preprocess_data/207946_classification_16s_barcode-v1 (1).csv', encoding = 'utf-8')

############################
# how many diferrent barcode

raw['barcode'].unique()

array(['BC04', nan, 'BC02', 'BC05', 'BC01', 'BC08', 'BC07', 'BC11',
       'BC12', 'BC06', 'BC09'], dtype=object)

In [50]:
#################################################
# Select raw data 
# (1) 'exit_status' == 'Classification successful'
# (2) 'barcode' == BC01, BC02, BC03, BC04

raw = pd.read_csv('../nanopore_preprocess_data/207946_classification_16s_barcode-v1 (1).csv', encoding = 'utf-8')



#####
#BC04
#####
raw_sucess_BC04 = raw[(raw['exit_status'] == 'Classification successful') & (raw['barcode'] == 'BC04')]

raw_sucess_BC04.to_csv('../191018result/raw_sucess_BC04.csv', encoding='utf-8')
print('BC04 read count:',len(raw_sucess_BC04))


BC04 read count: 736862


In [51]:
#####################
# BC04 groupby taxid

taxid_list_bc04 = pd.DataFrame(raw_sucess_BC04.groupby('taxid').count()['read_id'])

taxid_list_bc04.columns = ['read_count']

taxid_list_bc04.to_csv('../191018result/taxid_list_bc04.csv', encoding='utf-8')
taxid_list_bc04.head()


,read_count
taxid,
2,6294
89,1
126,1
154,1
292,1


### Part 2: Analysis 16s

### No Selection in accuracy

In [52]:
##############
# BC04 lineage
##############

############
# get taxid 

taxid04 = list(taxid_list_bc04.index)
read_count04 = list(taxid_list_bc04['read_count'])
total_count04 = len(raw_sucess_BC04)


##########################################################
# Create id_lineage dataframe to match the id with lineage

id_lineage04 = pd.DataFrame(columns= ['taxid'])
id_lineage04['taxid'] = taxid04
id_lineage04.head()

###############################################
# the lineage of id add to id_lineage dataframe

for i in range(len(taxid04)):
    L = ncbi.get_lineage(taxid04[i])
    
    for node in L:
        
        rank = ncbi.get_rank([node])
        id_lineage04.at[i, list(rank.values())[0]] = str(node)
        

id_lineage04.head()

/Users/hou/anaconda3/lib/python3.6/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 140100 was translated into 666
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))


,taxid,no rank,superkingdom,phylum,class,order,genus,species,family,species group,subspecies,subphylum,species subgroup,suborder
0,2,131567,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89,224471,2,1224,28216,80840,88,89,NaN,NaN,NaN,NaN,NaN,NaN
2,126,1783257,2,203682,203683,112,NaN,NaN,126,NaN,NaN,NaN,NaN,NaN
3,154,131567,2,203691,203692,136,146,154,137,NaN,NaN,NaN,NaN,NaN
4,292,131567,2,1224,28216,80840,32008,292,119060,87882,NaN,NaN,NaN,NaN


In [53]:
###############################
# BC04 superkingdom: Read count 

superkingdom = pd.DataFrame(list(zip(taxid04, id_lineage04['superkingdom'], read_count04)), 
                            columns=['taxid', 'superkingdom', 'read_count'] )


count_superkingdom = pd.DataFrame(superkingdom.groupby('superkingdom')['read_count'].sum())
count_superkingdom.insert(0, 'name', '')


name = []
for i in count_superkingdom.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_superkingdom['name'] = name
count_superkingdom = count_superkingdom.sort_values(by=['read_count'], ascending=False)

count_superkingdom['percentage'] = (count_superkingdom['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_superkingdom.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_superkingdom.sum(axis = 0, skipna = True)[2]

count_superkingdom.to_csv('../191018result/BC04_count_superkingdom_noSelect.csv', encoding='utf-8')

count_superkingdom = count_superkingdom.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_superkingdom.to_csv('../191018result/BC04_count_superkingdom_noSelect_unclassified.csv', encoding='utf-8')

count_superkingdom


,name,read_count,percentage
0,Bacteria,736859,99.999593
1,Unclassified,3,0.000407


In [54]:
#########################
# BC04 phylum: Read count 

phylum = pd.DataFrame(list(zip(taxid04, id_lineage04['phylum'], read_count04)), 
                            columns=['taxid', 'phylum', 'read_count'] )


count_phylum = pd.DataFrame(phylum.groupby('phylum')['read_count'].sum())
count_phylum.insert(0, 'name', '')


name = []
for i in count_phylum.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_phylum['name'] = name
count_phylum = count_phylum.sort_values(by=['read_count'], ascending=False)

count_phylum['percentage'] = (count_phylum['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_phylum.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_phylum.sum(axis = 0, skipna = True)[2]

count_phylum.to_csv('../191018result/BC04_count_phylum_noSelect.csv', encoding='utf-8')

count_phylum = count_phylum.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_phylum.to_csv('../191018result/BC04_count_phylum_noSelect_unclassified.csv', encoding='utf-8')

count_phylum

,name,read_count,percentage
0,Bacteroidetes,391769,53.167214
1,Firmicutes,136609,18.539292
2,Proteobacteria,93084,12.632487
3,Verrucomicrobia,54557,7.403964
4,Fusobacteria,54427,7.386322
5,Actinobacteria,24,0.003257
6,Cyanobacteria,2,0.000271
7,Spirochaetes,2,0.000271
8,Tenericutes,2,0.000271
9,Planctomycetes,1,0.000136


In [55]:
########################
# BC04 class: Read count 

class_ = pd.DataFrame(list(zip(taxid04, id_lineage04['class'], read_count04)), 
                            columns=['taxid', 'class', 'read_count'] )


count_class = pd.DataFrame(class_.groupby('class')['read_count'].sum())
count_class.insert(0, 'name', '')


name = []
for i in count_class.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_class['name'] = name
count_class = count_class.sort_values(by=['read_count'], ascending=False)

count_class['percentage'] = (count_class['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_class.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_class.sum(axis = 0, skipna = True)[2]

count_class.to_csv('../191018result/BC04_count_class_noSelect.csv', encoding='utf-8')

count_class = count_class.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_class.to_csv('../191018result/BC04_count_class_noSelect_unclassified.csv', encoding='utf-8')

count_class

,name,read_count,percentage
0,Bacteroidia,391545,53.136815
1,Gammaproteobacteria,78028,10.589228
2,Negativicutes,72178,9.795321
3,Clostridia,61881,8.397909
4,Verrucomicrobiae,54535,7.400979
5,Fusobacteriia,54427,7.386322
6,Betaproteobacteria,14355,1.948126
7,Bacilli,1040,0.141139
8,Deltaproteobacteria,587,0.079662
9,Erysipelotrichia,228,0.030942


In [56]:
########################
# BC04 order: Read count 

order = pd.DataFrame(list(zip(taxid04, id_lineage04['order'], read_count04)), 
                            columns=['taxid', 'order', 'read_count'] )


count_order = pd.DataFrame(order.groupby('order')['read_count'].sum())
count_order.insert(0, 'name', '')

name = []
for i in count_order.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_order['name'] = name
count_order = count_order.sort_values(by=['read_count'], ascending=False)

count_order['percentage'] = (count_order['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_order.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_order.sum(axis = 0, skipna = True)[2]

count_order.to_csv('../191018result/BC04_count_order_noSelect.csv', encoding='utf-8')

count_order = count_order.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_order.to_csv('../191018result/BC04_count_order_noSelect_unclassified.csv', encoding='utf-8')

count_order


,name,read_count,percentage
0,Bacteroidales,391461,53.125416
1,Enterobacterales,75774,10.283337
2,Clostridiales,61873,8.396823
3,Verrucomicrobiales,54535,7.400979
4,Fusobacteriales,54427,7.386322
5,Acidaminococcales,46896,6.364285
6,Burkholderiales,14064,1.908634
7,Lactobacillales,990,0.134354
8,Desulfovibrionales,580,0.078712
9,Aeromonadales,230,0.031213


In [57]:
#########################
# BC04 family: Read count 

family = pd.DataFrame(list(zip(taxid04, id_lineage04['family'], read_count04)), 
                            columns=['taxid', 'family', 'read_count'] )


count_family = pd.DataFrame(family.groupby('family')['read_count'].sum())
count_family.insert(0, 'name', '')

name = []
for i in count_family.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_family['name'] = name
count_family = count_family.sort_values(by=['read_count'], ascending=False)

count_family['percentage'] = (count_family['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_family.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_family.sum(axis = 0, skipna = True)[2]

count_family.to_csv('../191018result/BC04_count_family_noSelect.csv', encoding='utf-8')

count_family = count_family.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_family.to_csv('../191018result/BC04_count_family_noSelect_unclassified.csv', encoding='utf-8')

count_family


,name,read_count,percentage
0,Bacteroidaceae,280023,38.002095
1,Prevotellaceae,82791,11.235618
2,Fusobacteriaceae,54421,7.385508
3,Akkermansiaceae,53798,7.300960
4,Enterobacteriaceae,53722,7.290646
5,Lachnospiraceae,49422,6.707090
6,Acidaminococcaceae,46896,6.364285
7,Tannerellaceae,18954,2.572259
8,Sutterellaceae,11953,1.622149
9,Ruminococcaceae,5836,0.792007


In [58]:
########################
# BC04 genus: Read count 

genus = pd.DataFrame(list(zip(taxid04, id_lineage04['genus'], read_count04)), 
                            columns=['taxid', 'genus', 'read_count'] )


count_genus = pd.DataFrame(genus.groupby('genus')['read_count'].sum())
count_genus.insert(0, 'name', '')

name = []
for i in count_genus.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_genus['name'] = name
count_genus = count_genus.sort_values(by=['read_count'], ascending=False)

count_genus['percentage'] = (count_genus['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_genus.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_genus.sum(axis = 0, skipna = True)[2]

count_genus.to_csv('../191018result/BC04_count_genus_noSelect.csv', encoding='utf-8')

count_genus = count_genus.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_genus.to_csv('../191018result/BC04_count_genus_noSelect_unclassified.csv', encoding='utf-8')

count_genus


,name,read_count,percentage
0,Bacteroides,279409,37.918769
1,Prevotella,72074,9.781207
2,Fusobacterium,54050,7.335159
3,Akkermansia,53798,7.300960
4,Lachnoclostridium,32599,4.424031
5,Parabacteroides,18682,2.535346
6,Klebsiella,13141,1.783373
7,Sutterella,9562,1.297665
8,Anaerotignum,7156,0.971145
9,Citrobacter,4227,0.573649


In [59]:
##########################
# BC04 species: Read count 

species = pd.DataFrame(list(zip(taxid04, id_lineage04['species'], read_count04)), 
                            columns=['taxid', 'species', 'read_count'] )


count_species = pd.DataFrame(species.groupby('species')['read_count'].sum())
count_species.insert(0, 'name', '')

name = []
for i in count_species.index.values:
    
    name.append(list(ncbi.get_taxid_translator([i]).values())[0])
    
    
count_species['name'] = name
count_species = count_species.sort_values(by=['read_count'], ascending=False)

count_species['percentage'] = (count_species['read_count'] / total_count04)*100

unclassified_count = total_count04 - count_species.sum(axis = 0, skipna = True)[1]
unclassified_per = 100 - count_species.sum(axis = 0, skipna = True)[2]

count_species.to_csv('../191018result/BC04_count_species_noSelect.csv', encoding='utf-8')

count_species = count_species.append({"name": "Unclassified", "read_count": unclassified_count,"percentage":unclassified_per}, ignore_index=True)

count_species.to_csv('../191018result/BC04_count_species_noSelect_unclassified.csv', encoding='utf-8')

count_species


,name,read_count,percentage
0,Bacteroides plebeius,105613,14.332806
1,Prevotella stercorea,71400,9.689738
2,Bacteroides vulgatus,55928,7.590024
3,Bacteroides stercoris,55393,7.517418
4,Akkermansia muciniphila,53781,7.298653
5,Fusobacterium varium,45323,6.150812
6,Bacteroides caccae,25110,3.407694
7,[Clostridium] bolteae,23250,3.155272
8,Klebsiella pneumoniae,11649,1.580893
9,Parabacteroides distasonis,11397,1.546694


### Part 3: Species Diversity

In [62]:
import ecopy as ep
ep.load_data('varespec')
#varespec = ep.load_data('varespec')
#shannonH = ep.diversity(varespec, 'shannon')

ModuleNotFoundError: No module named 'ecopy'

In [60]:
diversity(count_species, method='shannon', breakNA=True, num_equiv=True)

NameError: name 'diversity' is not defined